In [32]:
import io
import numpy as np
import pandas as pd

import inspect

from mintalib import core

from mintalib import testing

from functools import lru_cache

from itertools import product

import talib

from talib import abstract

import warnings

import pandas as pd


In [33]:
dir(core)

['ADX',
 'ATR',
 'ATRL',
 'ATRP',
 'CURVE',
 'CURVE_BADOPTION',
 'CURVE_INTERCEPT',
 'CURVE_MSE',
 'CURVE_RMSE',
 'CURVE_RSQUARE',
 'CURVE_RVALUE',
 'CURVE_SLOPE',
 'CURVE_VALUE',
 'CurveOption',
 'DEMA',
 'DataFrame',
 'EMA',
 'EVAL',
 'Index',
 'Indicator',
 'KAMA',
 'MACD',
 'PPO',
 'PRICE',
 'PSAR',
 'RMA',
 'ROC',
 'RSI',
 'SLOPE',
 'SLOPE_BADOPTION',
 'SLOPE_CORRELATION',
 'SLOPE_DEFAULT',
 'SLOPE_FORECAST',
 'SLOPE_INTERCEPT',
 'SLOPE_RMSERROR',
 'SLOPE_VALUE',
 'SMA',
 'STDEV',
 'Series',
 'SlopeOption',
 'TEMA',
 'TRADE',
 'VOLUME',
 'WMA',
 '__all__',
 '__builtins__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__pyx_unpickle_Enum',
 '__pyx_unpickle___Pyx_EnumMeta',
 '__spec__',
 '__test__',
 'calc_adx',
 'calc_atr',
 'calc_curve',
 'calc_dema',
 'calc_ema',
 'calc_kama',
 'calc_log',
 'calc_macd',
 'calc_mid',
 'calc_ppo',
 'calc_price',
 'calc_psar',
 'calc_rma',
 'calc_roc',
 'calc_rsi',
 'calc_slope',
 'calc_sma',
 'calc_stdev',
 'calc_streak',


In [34]:
help(core.calc_ema)

Help on cython_function_or_method in module mintalib.core:

calc_ema(series, period=20)
    Exponential Moving Average
    
    Args:
        series (series) : The input series. Required
        period (int) : The indicator period. Default 20
    
    Returns:
        A series



In [35]:
help(core.SMA)

Help on class SMA in module mintalib.core:

class SMA(mintalib.model.Indicator)
 |  SMA(period=50, *, item=None)
 |  
 |  Simple Moving Average
 |  
 |  Method resolution order:
 |      SMA
 |      mintalib.model.Indicator
 |      abc.ABC
 |      mintalib.model.ReprMixin
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, period=50, *, item=None)
 |  
 |  calc(self, data)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset()
 |  
 |  same_scale = True
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from mintalib.model.Indicator:
 |  
 |  __call__(self, data)
 |      Call self as a function.
 |  
 |  __matmul__(self, other)
 |  
 |  __or__(self, other)
 |  
 |  calc_cached(self, data)
 |      calc with builtin caching
 |  
 |  clone(self, **kwargs)
 |  
 |  get_series(self, data)
 |  
 |  -------

In [36]:
inspect.signature(core.SMA.calc)


<Signature (self, data)>

In [37]:
prices = testing.get_prices()
type(prices)

pandas.core.frame.DataFrame

In [39]:
core.calc_atr(prices)


0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
245    1.909249
246    1.895731
247    1.891750
248    1.943768
249    1.920642
Length: 250, dtype: float64

In [20]:
core.calc_macd(prices.close)

,macd,macdsignal,macdhist
date,,,
2021-07-29,NaN,NaN,NaN
2021-07-30,NaN,NaN,NaN
2021-08-02,NaN,NaN,NaN
2021-08-03,NaN,NaN,NaN
2021-08-04,NaN,NaN,NaN
...,...,...,...
2022-07-21,-1.394736,-1.165696,-0.229041
2022-07-22,-1.639678,-1.260492,-0.379185
2022-07-25,-1.578367,-1.324067,-0.254300


In [21]:
names = [n[5:] for n in dir(core) if n.startswith("calc_")]
names


['adx',
 'atr',
 'curve',
 'dema',
 'ema',
 'kama',
 'log',
 'macd',
 'mid',
 'ppo',
 'price',
 'psar',
 'rma',
 'roc',
 'rsi',
 'slope',
 'sma',
 'stdev',
 'streak',
 'tema',
 'trange',
 'wma']

In [22]:

map_names = dict(mid='midprice')


def get_calc(name):
    func = getattr(core, f"calc_{name}", None)
    return func


def ref_ema(series, period):
    return series.ewm(span=period, min_periods=period, adjust=False, ignore_na=True).mean()


def ref_sma(series, period):
    return series.rolling(window=period).mean()


def get_ref(name):
    func = globals().get(f"ref_{name}")
    if func is None:
        func = getattr(abstract, name.upper(), None)
    return func


def get_first_param(func):
    signature = inspect.signature(func)
    parameters = list(signature.parameters.values())
    if parameters:
        param = parameters[0]
        return param.name



@lru_cache
def series_inputs():
    inputs = dict()
    for count, skip in product([50], [0, 20, -5]):
        key = f"S{count}:{skip}"
        data = random_walk(count, skip=skip)
        inputs[key] = data
    return inputs

@lru_cache
def prices_inputs():
    inputs = dict()
    for count, skip in product([50], [0, 20, -5]):
        key = f"P{count}:{skip}"
        data = sample_prices(count, skip=skip)
        inputs[key] = data
    return inputs


def test_name(name, period):
    calc = get_calc(name)
    ref = get_ref(name)

    if calc is None:
        warnings.warn("Calc for {name} not found!")
        return

    if ref is None:
        warnings.warn(f"Ref for {name} fnot found!")
        return

    param = get_first_param(calc)
    print(f"calc_{name}", param)

    if param == 'prices':
        inputs = prices_inputs()
    else:
        inputs = series_inputs()

    for key, data in inputs.items():
        print(name, key, period)
        result = calc(data, period)
        target = ref(data, period)
        compare = compare_results(result, target)

        print(name, type(result), type(target), compare)


def compare_results(result, target):
    if isinstance(result, pd.Series):
        result = result.values

    if isinstance(target, pd.Series):
        target = target.values

    return np.allclose(result, target, equal_nan=True)



test_name("sma", 20)
test_name("ema", 20)



calc_sma series
sma S50:0 20
sma <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'> True
sma S50:20 20
sma <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'> True
sma S50:-5 20
sma <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'> False
calc_ema series
ema S50:0 20
ema <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'> False
ema S50:20 20
ema <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'> False
ema S50:-5 20
ema <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'> False


In [23]:
data = """\
date          open    high     low   close    volume
2022-05-19   99.76  100.72   99.76  100.72  136654.0
2022-05-20  100.67  102.90  100.67  102.90  191287.0
2022-05-23  103.82  104.16  103.63  103.63  153282.0
2022-05-24  102.04  103.44  102.04  103.44  174975.0
2022-05-25  103.78  104.34  103.49  104.25  150746.0
2022-05-26  104.28  105.87  104.28  105.87   95146.0
2022-05-27  105.86  106.15  105.41  106.15  148301.0
2022-05-30  108.14  108.14  106.20  106.49  169159.0
2022-05-31  106.78  106.78  105.39  105.80  111953.0
2022-06-01  106.55  107.42  105.32  107.42  219275.0
2022-06-02  106.86  106.86  104.98  104.98  157016.0
2022-06-03  104.82  105.15  103.92  103.92  178344.0
2022-06-06  103.27  103.27  102.16  102.85  111618.0
2022-06-07  103.35  103.35  102.10  102.53  110751.0
2022-06-08  103.06  104.92  102.67  103.57   99175.0
2022-06-09  102.58  103.74  102.58  102.67  177273.0
2022-06-10  102.39  102.39  101.95  102.12  116301.0
2022-06-13  102.72  104.11  102.61  104.11   82395.0
2022-06-14  104.38  106.13  104.36  106.13  156676.0
2022-06-15  106.16  106.16  105.46  105.46  178853.0
2022-06-16  105.36  106.75  105.36  106.75  149042.0
2022-06-17  106.83  107.67  106.83  107.35  130764.0
2022-06-20  106.81  106.81  106.62  106.81  162909.0
2022-06-21  106.59  107.14  106.21  107.14  133974.0
2022-06-22  107.49  107.49  106.05  106.05  161829.0
2022-06-23  106.43  106.43  105.06  105.06  126359.0
2022-06-24  104.91  105.75  104.70  105.75  125444.0
2022-06-27  106.32  106.32  105.26  105.89  125450.0
2022-06-28  105.50  106.89  105.50  106.89  129627.0
2022-06-29  106.48  107.18  106.03  106.03  118273.0
2022-06-30  106.34  106.34  105.93  106.09  123633.0
2022-07-01  107.60  108.45  107.26  108.45  140498.0
2022-07-04  108.84  109.58  108.84  109.58  165658.0
2022-07-05  109.28  110.87  109.28  109.57  151093.0
2022-07-06  110.36  112.30  110.36  112.26  133831.0
2022-07-07  111.55  111.55  110.14  110.14  146914.0
2022-07-08  110.46  111.15  110.41  110.69  151584.0
2022-07-11  110.91  110.91  109.58  110.82  125386.0
2022-07-12  110.10  110.10  109.00  109.00  130852.0
2022-07-13  108.91  108.91  107.92  108.16  168689.0
2022-07-14  109.18  109.87  109.17  109.84  160095.0
2022-07-15  109.92  109.92  108.75  109.69  185769.0
2022-07-18  111.00  111.60  110.81  110.81  144026.0
2022-07-19  111.01  111.01  109.21  109.21  137981.0
2022-07-20  110.82  111.37  110.40  110.40  119492.0
2022-07-21  109.75  109.82  108.80  108.80  127867.0
2022-07-22  109.27  109.27  107.64  107.64  182014.0
2022-07-25  107.94  107.94  105.99  105.99  157951.0
2022-07-26  107.50  108.11  107.19  107.40  191611.0
2022-07-27  108.38  108.80  107.96  108.80  114188.0
"""

In [12]:
pd.read_csv(data)

FileNotFoundError: [Errno 2] No such file or directory: 'date          open    high     low   close    volume\n2022-05-19   99.76  100.72   99.76  100.72  136654.0\n2022-05-20  100.67  102.90  100.67  102.90  191287.0\n2022-05-23  103.82  104.16  103.63  103.63  153282.0\n2022-05-24  102.04  103.44  102.04  103.44  174975.0\n2022-05-25  103.78  104.34  103.49  104.25  150746.0\n2022-05-26  104.28  105.87  104.28  105.87   95146.0\n2022-05-27  105.86  106.15  105.41  106.15  148301.0\n2022-05-30  108.14  108.14  106.20  106.49  169159.0\n2022-05-31  106.78  106.78  105.39  105.80  111953.0\n2022-06-01  106.55  107.42  105.32  107.42  219275.0\n2022-06-02  106.86  106.86  104.98  104.98  157016.0\n2022-06-03  104.82  105.15  103.92  103.92  178344.0\n2022-06-06  103.27  103.27  102.16  102.85  111618.0\n2022-06-07  103.35  103.35  102.10  102.53  110751.0\n2022-06-08  103.06  104.92  102.67  103.57   99175.0\n2022-06-09  102.58  103.74  102.58  102.67  177273.0\n2022-06-10  102.39  102.3

In [24]:
pd.read_csv(io.StringIO(data), sep='\s+', parse_dates=True)

,date,open,high,low,close,volume
0,2022-05-19,99.76,100.72,99.76,100.72,136654.0
1,2022-05-20,100.67,102.90,100.67,102.90,191287.0
2,2022-05-23,103.82,104.16,103.63,103.63,153282.0
3,2022-05-24,102.04,103.44,102.04,103.44,174975.0
4,2022-05-25,103.78,104.34,103.49,104.25,150746.0
5,2022-05-26,104.28,105.87,104.28,105.87,95146.0
6,2022-05-27,105.86,106.15,105.41,106.15,148301.0
7,2022-05-30,108.14,108.14,106.20,106.49,169159.0
8,2022-05-31,106.78,106.78,105.39,105.80,111953.0
9,2022-06-01,106.55,107.42,105.32,107.42,219275.0
